Work-Flow:

1. Voacbulary mapping each word to a index
2. setup a pytorch dataset to load the data
3. setup padding of every batch( all example should be of same seq_length and setup dataloader)


In [3]:
import os  # loading file path
from pathlib import Path
import pandas as pd
import spacy  # for tokenizer
import torch
from torch.nn.utils.rnn import pad_sequence  # pad batch
from torch.utils.data import DataLoader, Dataset
from PIL import Image  # load image
import torchvision.transforms as transforms

In [5]:
current_dir = Path.cwd().parent
IMAGE_DIR = current_dir / "data" / "flickr8k" / "images"
CAPTION_DIR = current_dir / "data" / "flickr8k" / "captions.txt"

spacy_en = spacy.load("en_core_web_sm")

In [8]:
class Vocabulary:
    def __init__(self, freq_threshold):
        self.itos = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.stoi = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.freq_threshold = freq_threshold

    def __len__(self):
        return len(self.itos)

    @staticmethod
    def tokenizer_eng(text):
        return [tok.text.lower() for tok in spacy_en.tokenizer(text)]

    def build_vocabulary(self, sentence_list):
        frequencies = {}
        idx = 4
        for sentence in sentence_list:
            for word in self.tokenizer_eng(sentence):
                if word not in frequencies:
                    frequencies[word] = 1
                else:
                    frequencies[word] += 1
                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1

    def numericalize(self, text):
        tokenize_text = self.tokenizer_eng(text)
        return [
            self.stoi[token] if token in self.stoi else self.stoi["UNK"]
            for token in tokenize_text
        ]

In [9]:
class FlickrDataset(Dataset):
    def __init__(self, root_dir, caption_file, transform=None, freq_threshold=5):
        self.root_dir = root_dir
        self.df = pd.read_csv(caption_file)
        self.transform = transform

        # get image, caption
        self.img = self.df["image"]
        self.captions = self.df["caption"]

        # initialize vocabulary and build vocabulary
        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocabulary(self.captions.tolist())

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        caption = self.captions[index]
        img_id = self.img[index]
        img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")

        if self.transform is not None:
            img = self.transform(img)

        numericalized_caption = [self.vocab.stoi["<SOS>"]]
        numericalized_caption += self.vocab.numericalize(caption)
        numericalized_caption.append(self.vocab.stoi["<EOS>"])

In [10]:
class MyCollate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx

    def __call__(self, batch):
        imgs = [
            item[0].squeeze(0) for item in batch
        ]  #  remove dimensions with size 1 from a tensor
        imgs = torch.cat(imgs, dim=0)
        targets = [item[1] for item in batch]
        targets = pad_sequence(targets, batch_first=False, padding_value=self.pad_idx)

        return imgs, targets

In [11]:
def get_loader(
    root_folder,
    annotation_file,
    transform,
    batch_size=32,
    num_workers=4,
    shuffle=True,
    pin_memory=True,
):
    dataset = FlickrDataset(root_folder, annotation_file, transform)
    pad_idx = dataset.vocab.stoi["<PAD>"]

    loader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=True,
        pin_memory=pin_memory,
        collate_fn=MyCollate(pad_idx=pad_idx),
    )
    return loader

In [12]:
transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]
)
dataloader = get_loader(
    root_folder=IMAGE_DIR, annotation_file=CAPTION_DIR, transform=transform
)
for idx, (imgs, captions) in enumerate(dataloader):
    print(imgs.shape)
    print(captions.shape)

In [ ]:
nlp = spacy.load("en_core_web_sm")

# Input text
text = "This is an Example sentence for tokenization."

# Tokenize and lowercase using a list comprehension
lowercase_tokens = []
for tok in nlp(text):
    print(tok.text.lower())
    lowercase_tokens.append(tok)

# Print the lowercase tokens
print([tok.text.lower() for tok in nlp(text)])

this
is
an
example
sentence
for
tokenization
.
['this', 'is', 'an', 'example', 'sentence', 'for', 'tokenization', '.']


In [ ]:
stoi = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
numericalized_caption = [stoi["<SOS>"]]
print(stoi)
print(numericalized_caption)

{'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3}
[1]
